## Approach

We will be looking for nested squares with big areas inside them. To do that we firstly apply some filters (bilateral and Gaussian), extract edges with Canny filter. After that we find contours and their hierarchy to be able to find parents and children in terms of nesting.

In [2]:
import cv2
import numpy as np
from tqdm.notebook import tqdm
from time import time
import os

DATA_PATH = '../data/input/'
RESULT_PATH = '../data/result/'

In [4]:
def children_count(hierarchy, parent, inner=False):
    if parent == -1:
        return 0
    elif not inner:
        return children_count(hierarchy, hierarchy[parent][2], True)
    return children_count(hierarchy, hierarchy[parent][0], True) + children_count(hierarchy, hierarchy[parent][2], True) + 1

def has_parent_square(hierarchy, squares, parent):
    if hierarchy[parent][3] == -1:
        return False
    if hierarchy[parent][3] in squares:
        return True
    return has_parent_square(hierarchy, squares, hierarchy[parent][3])

def detect_finder_patterns(image: np.ndarray, length: float = 0.15, area: float = 1000.0):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    filtered_image = cv2.bilateralFilter(gray_image, 11, 17, 17)
    blured_image = cv2.GaussianBlur(filtered_image, (3, 3), 0)
    edged_image = cv2.Canny(blured_image, 30, 200)

    contours, hierarchy = cv2.findContours(edged_image.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    squares = []
    square_indices = []
    
    for i, contour in enumerate(contours):
        contour_length = cv2.arcLength(contour, True)
        contour_area = cv2.contourArea(contour)
        approx_contour = cv2.approxPolyDP(contour, 0.03 * contour_length, True)

        if len(approx_contour) == 4:
            children = children_count(hierarchy[0], i)
            has_parent = has_parent_square(hierarchy[0], square_indices, i)
            
            if contour_area > area and np.abs(((contour_length / 4) ** 2) / contour_area - 1) < length and children > 1 and not has_parent:
                squares.append(approx_contour)
                square_indices.append(i)

    cv2.drawContours(image, squares, -1, (0, 0, 255), 5)

    return image

start = time()

for test_suite in (1, 2, 3):
    test_suite_path = os.path.join(DATA_PATH, f"TestSet{test_suite}")
    result_suite_path = os.path.join(RESULT_PATH, f"TestSet{test_suite}Result")
    if not os.path.exists(result_suite_path):
        os.mkdir(result_suite_path)
    for image_name in tqdm(os.listdir(test_suite_path)):
        image = cv2.imread(os.path.join(test_suite_path, image_name))
        preprocessed_image = detect_finder_patterns(image)
        cv2.imwrite(os.path.join(result_suite_path, image_name), preprocessed_image)

print("Time elapsed:", time() - start)

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

Time elapsed: 142.6408452987671


## Results

### Quality

| Dataset | Precision/Recall |
|:-------:|:----------------:|
|TestSet1| 0.86/0.9 |
|TestSet2| 0.85/0.8 |
|TestSet3| 0.58/0.75 |

### Time

Average working time on test datesets is 0.35 seconds per image.